In [1]:
from astropy.io import fits
import astropy.units as u
from astroquery import ned
from astropy.wcs import wcs
from astropy.coordinates import SkyCoord
from astropy.table import Table, vstack, hstack
import numpy as np
import tables
from tabulate import tabulate
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from math import ceil
import pandas as pd

In [2]:
from tqdm import tqdm

In [3]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [4]:
# This is a cell with all self-defined functions
def coord_NED(dwarfname):
    from astroquery.ned.core import RemoteServiceError
    # get rid of the weird symbos *, (, ), etc, since NED doens't like it
    newname = dwarfname.replace('*','').replace('(','').replace(')', '')

    try:
        target = ned.Ned.query_object(newname)
        ned_exist = True
    except RemoteServiceError: 
        print('NED does not have info, skip: %s/%s\n'%(dwarfname, newname))
        ned_exist= False 
    
    if ned_exist == False:
        tRA, tDEC, tVel, tDist = np.nan, np.nan, np.nan, np.nan
    else: # if NED does have this target, then extract coordinates
        tRA = target["RA(deg)"].data.data[0]
        tDEC = target["DEC(deg)"].data.data[0]
        tVel = target["Velocity"].data.data[0]
        tDist = target["Distance (arcmin)"].data.data[0]
    return tRA, tDEC, tDist, tVel, ned_exist

## transform from vhelio to vlsr 
def vhelio2vlsr_Westmeier(vel_init, ral, decb, reverse=False, doradec=True):
    '''
    - from http://www.atnf.csiro.au/people/Tobias.Westmeier/tools_hihelpers.php
    - obj_ra:  should be in degree
    - obj_dec: should be in degree
    - vel_init: velocity that need to be transformed
    -          vel_init = vhelio if reverse = False
    -          vel_init = vlsr   if reverse = True
    - Favor this one than the other one from Rosolowsky
    '''
    from astropy.coordinates import SkyCoord
    import numpy as np

    if doradec==True:
        c = SkyCoord(ral, decb, unit='deg')
        l = np.radians(c.galactic.l.value)
        b = np.radians(c.galactic.b.value)
    else:
        l = np.radians(ral)
        b = np.radians(decb)
    # vlsr 00> vhelio
    if reverse:
        delv = -(9*np.cos(l)*np.cos(b)+12*np.sin(l)*np.cos(b)+7*np.sin(b))
    else:
        delv = +9*np.cos(l)*np.cos(b)+12*np.sin(l)*np.cos(b)+7*np.sin(b)

    # print 'Velocity correction at this (RA, DEC) is (km/s): ', delv
    return vel_init+delv


def find_the_cube(ra, dec, observation='HI4PI'):
    '''
    Use the input (ra, dec) to decide which cube the data locates.

    observation: most of the time, it is HI4PI or GALFA-HI . 
    
    Note: GALFA-HI table is not available now, will do later 
    '''

    clt = Table.read('tables/%s_RADEC.dat'%(observation), format='ascii')
    cramin, cramax = clt['min_ra'], clt['max_ra']
    cdcmin, cdcmax = clt['min_dec'], clt['max_dec']
    indra = np.all([ra>cramin, ra<cramax], axis=0)
    inddc = np.all([dec>cdcmin, dec<cdcmax], axis=0)
    indall = np.where(np.all([indra, inddc], axis=0) == True)

    cubename = clt['cubename'][indall]
    if len(cubename)==0:
        return 'not found'
    else:
        cubename = cubename[0]

        return cubename
    
import sys
import astropy.wcs as wcs
import numpy as np


# a function to read in header and generate RA, DEC and VLSR array
def get_cubeinfo(header, returnHeader=False):
    '''
    A function created to parse the RA, DEC, (and velocity) information from the 2D (3D) header

    - This function has been tested with GALFA-HI/EBHIS cubes, and GALFA-HI 2D images.
    -               also been tested with LAB cubes/images that are in (glon, glat) coordinates.
    - The input header can be 2D: NAXIS=2. NAXIS1 is RA (glon), 2 is DEC (glat)
    -                      or 3D: NAXIS=3. NAXIS1 is RA (glon), 2 is DEC (glat), 3 is Velocity
    - Return: if GALFA-HI or EBHIS:
                        ra and dec in 2D, with shape of (dec.size, ra.size) or (NAXIS2, NAXIS1)
    -                   velocity in 1D.
                        or (ra, dec, vlsr, header array)
    -         if LAB:
                        glon, glat in 2D, with shape of (glat.size, glon.size) or (NAXIS2, NAXIS1)
    -                   velocity in 1D.
                        or (gl, gb, vlsr, header array)
    - History: updated as of 2016.10.03. Yong Zheng @ Columbia Astro.
    '''


    hdrarrs = []
    if header['NAXIS'] == 2:
        hdr2d = header.copy()
        hdrarrs.append(hdr2d)
    elif header['NAXIS'] == 3:
        # create a 2D header (RA/DEC) to speed up the RA/DEC calculation using astropy.wcs
        hdr2d = header.copy()
        # we don't need the velocity (3) information in the header
        delkey = []
        for key in hdr2d.keys():
            if len(key) != 0 and key[-1] == '3': delkey.append(key)
        for i in delkey: del hdr2d[i]

        hdr2d['NAXIS'] = 2
        if 'WCSAXES' in hdr2d.keys(): hdr2d['WCSAXES']=2
        # create a 1D header (vel) to parse the velocity using astropy.wcs
        hdr1d = header.copy()
        # we don't need the RA/DEC keywords info in the header now.
        delkey = []
        for keya in hdr1d.keys():
            if len(keya) != 0 and keya[-1] in ['1', '2']: delkey.append(keya)
        for i in delkey: del hdr1d[i]
        delkey = []
        for keyb in hdr1d.keys():
            if len(keyb) != 0 and keyb[-1] == '3':
                hdr1d.append('%s1'%(keyb[:-1]))
                hdr1d['%s1'%(keyb[:-1])] = hdr1d[keyb]
                delkey.append(keyb)
        for i in delkey: del hdr1d[i]
        hdr1d['NAXIS'] = 1
        if 'WCSAXES' in hdr1d.keys(): hdr1d['WCSAXES']=1

        # save header arrays
        hdrarrs.append(hdr2d)
        hdrarrs.append(hdr1d)
    else:
        print("This code can only handle 2D or 3D data")
        sys.exit(1)

    return_arrays = []

    # calculate RA, DEC
    gwcsa = wcs.WCS(hdr2d)
    n1, n2 = hdr2d['NAXIS1'], hdr2d['NAXIS2']
    ax = np.reshape(np.mgrid[0:n1:1]+1, (1, n1))  # For FITS standard, origin = 1
    ay = np.reshape(np.mgrid[0:n2:1]+1, (n2, 1))  #   then for numpy standard, origin = 0
    coor1, coor2 = gwcsa.all_pix2world(ax, ay, 1) # coor1 = ra  or glon
    return_arrays.append(coor1)                   # coor2 = dec or glat
    return_arrays.append(coor2)

    ## calculate VLSR
    if header['NAXIS'] == 3:
        gwcsb = wcs.WCS(hdr1d)
        n1 = hdr1d['NAXIS1']
        ax = np.mgrid[0:n1:1]+1
        # ax = np.linspace(0, n1, n1)  # nope, wrong
        vel = gwcsb.all_pix2world(ax, 1)[0]
        if 'CUNIT1' in hdr1d.keys():
            if hdr1d['CUNIT1'] in ['m/s', 'M/S', 'M/s', 'm/S']:
                vel = vel/1e3
        else: vel = vel/1e3  # default is usually in m/s
        return_arrays.append(vel)

    if returnHeader == True: return_arrays.append(hdrarrs)
    return return_arrays

In [5]:
def ellipse_mask_cube(semi_a, semi_b, PA, tar_x, tar_y, cube_header, extend_pix = 0):
    ## make ellipse based on the semi major/minor axies and position angle of the targets 
    
    cube_wh = cube_header['NAXIS1']  # RA
    cube_ht = cube_header['NAXIS2']  # DEC
    x1d = np.arange(cube_wh)
    y1d = np.arange(cube_ht)
    x2d, y2d = np.meshgrid(x1d, y1d)

    # center at the source 
    x2d_nc, y2d_nc = x2d-tar_x, y2d-tar_y

    # this is a rough pixel length for the semimajor and semiminor axis
    semi_a_pix = ceil(abs(semi_a / cube_header['CDELT1']))
    semi_b_pix = ceil(abs(semi_b / cube_header['CDELT1']))
    
    #phi = np.radians(90-PA) changing 
    phi = np.radians(90+PA)

    # transform to new coordinate system
    xx_prime = x2d_nc*np.cos(phi) + y2d_nc*np.sin(phi)
    yy_prime = -x2d_nc*np.sin(phi) + y2d_nc*np.cos(phi)

    mask_ep = ((xx_prime/(semi_a_pix+extend_pix))**2 + (yy_prime/(semi_b_pix+extend_pix))**2)<=1 
    return mask_ep

In [6]:
def find_info(objname):
    from astropy.table import Table 
    tb = Table.read('/Users/janagrc/repos/dwarfs/HIdwarflimits/dwarfgalcomplete.txt', format='ascii')
    
    newname = objname.replace('*','').replace('(','').replace(')', '').replace(' ', '')
    for i, iname in enumerate(tb['GalaxyName']):
        i_tbnewname = iname.replace('*','').replace('(','').replace(')', '').replace(' ', '')
        
        if newname == i_tbnewname:
            find_it = True
            break
        else: 
            find_it = False
            
    if find_it == True:
        print('YES, we find it!!!', tb['RA'][i], tb['DEC'][i], tb['vh(m/s)'][i])
        return tb['RA'][i], tb['DEC'][i], tb['vh(m/s)'][i], tb['rh(arcmins)'][i], tb['e=1-b/a'][i], tb['PA'][i]
    else:
        print('Sorry, not there, check your object name. ')
        return np.nan, np.nan, np.nan
    

# Galaxy Info

1 	SagittariusdSph 	(I don’t understand where the limit in GP or Mateo is from, Spekkens has <7880 from GASS, K94 has <10000) SKIP THERE ARE DEEPER LIMITS FOR THE CORE
6 	CetusII 	NEW not seeing velocity ref for this one
12 	ComaBerenices 	188+26s.mir stdev 0.04K so same limit of 0.02 Jy (towards edge, but looks clear, same with baseline but 0.2K source would be booming) 0.02*10*.236*44^2 = 100 Msun
13 	TucanaIV 	- not seeing anything for this one
15 	GrusII 	- not seeing one
17 	TucanaV 	- not seeing one, Conn et al. 2018 says chance could be grouping of stars assoc with SMC
19 	SagittariusII 	- not seeing anything,Laevens et al. 2015 discovery paper makes it sound like it could be a weird one again
23 	UrsaMinor 	NaN
24 	HorologiumII 	- no seeing, Kim & Jerjen 2015 discovery paper
26 	Eridanus3 	- not seeing, Conn et al. says maybe star cluster
27 	Phoenix2 	- not seeing
29 	Sextans(I) 	(too south for g)
30 	ReticulumIII 	- not seeing
31 	Indus1 	-
32 	UrsaMajor(I) 	NaN
34 	Pictoris1 	- not seeing
36 	Hercules 	244+10s stdev 0.08K
43 	ColumbaI 	- not velocity
44 	Pegasus3 	340+2s.mir - not seeing ~0.06K void 30 to -60 km/s about
45 	IndusI? 	- not seeing
46 	CanesVenatici(I) 	204+34 0.06K would benefit from unsharp mask as close to Galactic, baselines generally look quite good, but can see Josh’s point on velocity on-off on this one as baseline going up and down with position
60 	AndromedaXIX 	004+34, m31
66 	AndromedaXXXII 	- not seeing
68 	AndromedaXXX 	NaN
72 	AndromedaII 	020+34,m31?"\r\nNGC185 0 38 58.0 48 20 15 23.95 0.09 0.09 -203.8 1.1 1.1 0.11 m31\r\nAndromedaXXIX 23 58 55.6 30 45 20 24.32 0.22 0.22 -194.4 1.5 1.5 99.9 g31dr1 356+34"
76 	AndromedaXXXI 	- not seeing
82 	AndromedaXXXIII 	- not seeing
85 	Eridanus2 	- not seeing

In [7]:
NewDwarfs = pd.read_csv("./newdwarfs.txt", delim_whitespace=True ,header=(0))

In [8]:
NewDwarfs

,GalaxyName,ra1,ra2,ra3,dec1,dec2,dec3,ra,dec,EB-V,(m-M)o,e5,e6,vh(km/s),Notes
0,CetusIII,2.0,5.0,19.4,-4.0,16.0,12.0,31.3292,-4.2700,NaN,22.00,0.20,0.10,NaN,http://adsabs.harvard.edu/abs/2018PASJ...70S..18H
1,DESJ0225+0304,2.0,25.0,42.4,3.0,4.0,10.0,36.4250,3.0694,NaN,16.88,0.06,0.05,NaN,http://adsabs.harvard.edu/abs/2017MNRAS.468...97L
2,PictorII,6.0,44.0,43.2,-59.0,53.0,49.0,101.1792,-59.8969,0.107,18.30,0.12,0.15,NaN,http://adsabs.harvard.edu/abs/2016ApJ...833L...5D
3,VirgoI,12.0,0.0,9.6,0.0,40.0,48.0,180.0417,-0.6800,19.700,0.30,0.20,NaN,http://adsabs.harvard.edu/abs/2016ApJ...832...21H,NaN
4,HydrusI,2.0,29.0,33.3,-79.0,18.0,32.0,-0.6800,-79.3089,NaN,17.20,0.04,0.04,80.4,http://adsabs.harvard.edu/abs/2018MNRAS.479.5343K
5,CarinaII,NaN,NaN,NaN,NaN,NaN,NaN,114.1066,-57.9991,NaN,17.79,0.05,0.05,477.2,http://adsabs.harvard.edu/abs/2018MNRAS.475.5085T
6,CarinaIII,NaN,NaN,NaN,NaN,NaN,NaN,114.6298,-57.8997,NaN,17.22,0.10,0.10,284.6,http://adsabs.harvard.edu/abs/2018MNRAS.475.5085T
7,CraterII,NaN,NaN,NaN,NaN,NaN,NaN,177.3100,-18.4130,NaN,20.35,0.02,0.02,87.5,http://adsabs.harvard.edu/abs/2016MNRAS.459.2370T
8,AquariusII,NaN,NaN,NaN,NaN,NaN,NaN,338.4813,-9.3274,NaN,20.16,0.07,0.07,-71.1,http://adsabs.harvard.edu/abs/2016MNRAS.463..712T


In [ ]:
hi4pi_novelocity = ['CetusII','TucanaIV','GrusII','TucanaV','Sagittarius','HorologiumI','Eridanus3','Phoenix2','ReticulumII','Indus1','Pictoris1','ColumbaI','IndusI?','AndXXXII','AndXXXI','AndXXXIII','Eridanus2']


In [72]:
objname = 'CetusIII' 

objRA, objDEC, objV_helio, a, ecc, PA = find_info(objname)
vcoord = vhelio2vlsr_Westmeier(0, objRA, objDEC, doradec=True) #in km/s
objV = objV_helio + vcoord*1e3
semi_b = (1 - ecc)*a/60 #degree, semi minor axis
semi_a = a/60 #degree, semi major axis

#For unresolved undetected only looking at 30 arcmin region:  
semi_a = .25  
semi_b = .25

Sorry, not there, check your object name. 


ValueError: not enough values to unpack (expected 6, got 3)

In [47]:
#SKIP!
#Just to find the relevant cubes
for index, row in NewDwarfs.iterrows():
    objRA = row['ra']
    objDEC = row['dec']
#objRA, objDEC, objV_helio, a, ecc, PA = find_info(objname)
    cubename = find_the_cube(objRA, objDEC, observation='HI4PI')
    print(cubename)

CAR_E02
CAR_E02
CAR_B06
CAR_E09
not found
CAR_B06
CAR_B06
CAR_D09
CAR_D17


In [48]:
NewDwarfs

,GalaxyName,ra1,ra2,ra3,dec1,dec2,dec3,ra,dec,EB-V,(m-M)o,e5,e6,vh(km/s),Notes
0,CetusIII,2.0,5.0,19.4,-4.0,16.0,12.0,31.3292,-4.2700,NaN,22.00,0.20,0.10,NaN,http://adsabs.harvard.edu/abs/2018PASJ...70S..18H
1,DESJ0225+0304,2.0,25.0,42.4,3.0,4.0,10.0,36.4250,3.0694,NaN,16.88,0.06,0.05,NaN,http://adsabs.harvard.edu/abs/2017MNRAS.468...97L
2,PictorII,6.0,44.0,43.2,-59.0,53.0,49.0,101.1792,-59.8969,0.107,18.30,0.12,0.15,NaN,http://adsabs.harvard.edu/abs/2016ApJ...833L...5D
3,VirgoI,12.0,0.0,9.6,0.0,40.0,48.0,180.0417,-0.6800,19.700,0.30,0.20,NaN,http://adsabs.harvard.edu/abs/2016ApJ...832...21H,NaN
4,HydrusI,2.0,29.0,33.3,-79.0,18.0,32.0,-0.6800,-79.3089,NaN,17.20,0.04,0.04,80.4,http://adsabs.harvard.edu/abs/2018MNRAS.479.5343K
5,CarinaII,NaN,NaN,NaN,NaN,NaN,NaN,114.1066,-57.9991,NaN,17.79,0.05,0.05,477.2,http://adsabs.harvard.edu/abs/2018MNRAS.475.5085T
6,CarinaIII,NaN,NaN,NaN,NaN,NaN,NaN,114.6298,-57.8997,NaN,17.22,0.10,0.10,284.6,http://adsabs.harvard.edu/abs/2018MNRAS.475.5085T
7,CraterII,NaN,NaN,NaN,NaN,NaN,NaN,177.3100,-18.4130,NaN,20.35,0.02,0.02,87.5,http://adsabs.harvard.edu/abs/2016MNRAS.459.2370T
8,AquariusII,NaN,NaN,NaN,NaN,NaN,NaN,338.4813,-9.3274,NaN,20.16,0.07,0.07,-71.1,http://adsabs.harvard.edu/abs/2016MNRAS.463..712T


In [69]:
objname = 'TucanaIV' 

objRA, objDEC, objV_helio, a, ecc, PA = find_info(objname)
vcoord = vhelio2vlsr_Westmeier(0, objRA, objDEC, doradec=True) #in km/s
objV = objV_helio + vcoord*1e3
semi_b = (1 - ecc)*a/60 #degree, semi minor axis
semi_a = a/60 #degree, semi major axis

#For unresolved undetected only looking at 30 arcmin region:  
semi_a = .25  
semi_b = .25

YES, we find it!!! 0.7292 -60.85 999001


In [70]:
cubename = '/Volumes/MPPassport/amalya2/HI4PI/smoothnew/CAR_E02.fits' #pull correct cube from directory
img1 = fits.getdata(cubename)
hdr = fits.getheader(cubename)

In [71]:
# setting up the WCS
wcsleop =  wcs.WCS(hdr)

# converting to pixel space
tar_x, tar_y, tar_v = wcsleop.all_world2pix(objRA, objDEC, objV, 0)

tar_x = int(tar_x)
tar_y = int(tar_y)
tar_v = int(tar_v)

# & checking it's in there, make sure the shape fits the size of tar_v, tar_x, tar_y
img1.shape,  tar_v, tar_x, tar_y
#help(wcsleop.all_world2pix)

### get the whole RA/DEC/Vel channel from the header 
all_ra, all_dec, all_vlsr = get_cubeinfo(hdr)  # all_vlsr is in km/s

In [67]:
# figure out how large the area surrounding the source that we want 
ep1 = ellipse_mask_cube(semi_a, semi_b, PA, tar_x, tar_y, hdr, extend_pix=0) #30 arcmin diameter because already changed semi_a and semi_b initially
ep2 = ellipse_mask_cube(.5, .5, PA, tar_x, tar_y, hdr, extend_pix = 0) #1 degree diameter

# No Velocity:

In [56]:
#all_vlsr[tar_v] # the velocity where the target central velocity channel is 

In [68]:
def std_array(objname, datacube): 
    
    vel_bins = np.mgrid[-750:750:10]  # (-600, -590, .... 590, 600) bin size in velocity km/s
    #-600 to 601 for hi4pi, -750 to 751 for GALFA-HI
    flux_stds = np.zeros(vel_bins.size)
    
    for i, iv in enumerate(vel_bins):
        # find the closest velocity to iv 
        v_chan = np.argmin(np.fabs(all_vlsr-iv))
        #print(v_chan, iv, all_vlsr[v_chan])      # and test iv and all_vlsr[v_chan] are indeed the closest   
        f_std = np.nanstd(datacube[v_chan][ep1]) # ep1 is 30 arcmin beam, ep2 would be 1 degree
        flux_stds[i] = f_std
    return vel_bins, flux_stds

vel_bins, flux_stds = std_array(objname, img1)
# vel_bins, np.arange(all_vlsr.min(), all_vlsr.max(), 10)

std_array(objname, img1)

vel_bins

/Users/janagrc/.pyenv/versions/jupyter3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


array([-750, -740, -730, -720, -710, -700, -690, -680, -670, -660, -650,
       -640, -630, -620, -610, -600, -590, -580, -570, -560, -550, -540,
       -530, -520, -510, -500, -490, -480, -470, -460, -450, -440, -430,
       -420, -410, -400, -390, -380, -370, -360, -350, -340, -330, -320,
       -310, -300, -290, -280, -270, -260, -250, -240, -230, -220, -210,
       -200, -190, -180, -170, -160, -150, -140, -130, -120, -110, -100,
        -90,  -80,  -70,  -60,  -50,  -40,  -30,  -20,  -10,    0,   10,
         20,   30,   40,   50,   60,   70,   80,   90,  100,  110,  120,
        130,  140,  150,  160,  170,  180,  190,  200,  210,  220,  230,
        240,  250,  260,  270,  280,  290,  300,  310,  320,  330,  340,
        350,  360,  370,  380,  390,  400,  410,  420,  430,  440,  450,
        460,  470,  480,  490,  500,  510,  520,  530,  540,  550,  560,
        570,  580,  590,  600,  610,  620,  630,  640,  650,  660,  670,
        680,  690,  700,  710,  720,  730,  740])

In [59]:
rowv = flux_stds

np.nanmedian(flux_stds) #= med? 

/Users/janagrc/.pyenv/versions/jupyter3/lib/python3.6/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


nan

In [63]:
#median around: looking at by eye
med = .05
row_med = ([objname, med])

In [64]:
tbvel = Table(names = vel_bins)
tbvel.add_row(rowv)

tbmed = Table(names=('GalaxyName', 'med'), 
             meta={'name': 'first table'},
              dtype=('<U11', 'f8'))

tbmed.add_row(row_med)

tbvel
tb = hstack([tbmed, tbvel])

f = open('galfa_novelocity_jana.txt', 'w')
table = tb
f.write(tabulate(tb))
f.close

tb

GalaxyName,med,-750,-740,-730,-720,-710,-700,-690,-680,-670,-660,-650,-640,-630,-620,-610,-600,-590,-580,-570,-560,-550,-540,-530,-520,-510,-500,-490,-480,-470,-460,-450,-440,-430,-420,-410,-400,-390,-380,-370,-360,-350,-340,-330,-320,-310,-300,-290,-280,-270,-260,-250,-240,-230,-220,-210,-200,-190,-180,-170,-160,-150,-140,-130,-120,-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10,0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400,410,420,430,440,450,460,470,480,490,500,510,520,530,540,550,560,570,580,590,600,610,620,630,640,650,660,670,680,690,700,710,720,730,740
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
TucanaIV,0.05,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# Table

In [ ]:
tba = Table(names=('GalaxyName', 'med_std'), 
             meta={'name': 'first table'},
              dtype=('<U11', 'f8'))
tbnew.add_row(rowa)


#tb = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/hi4pi_resolved_undetected_table.txt', format='ascii')
#tb.add_row(rowd)

tb = (hstack([tba, tbb]))

f = open('galfa_unresolved_undetected_range.txt', 'w')
table = tb
f.write(tabulate(tb))
f.close

tb


# Std30_med, If Velocity Defined: 

In [ ]:
#a velocity channels, optical size of galaxy, this has been extended to 30arcmin for them all from the beginning 
a_1 = (img1[tar_v-5][ep1])
a_2 = (img1[tar_v-4][ep1])
a_3 = (img1[tar_v-3][ep1])
a_4 = (img1[tar_v-2][ep1])
a_5 = (img1[tar_v-1][ep1])
a_6 = (img1[tar_v][ep1])
a_7 = (img1[tar_v+1][ep1])
a_8 = (img1[tar_v+2][ep1])
a_9 = (img1[tar_v+3][ep1])
a_10 = (img1[tar_v+4][ep1])
a_11 = (img1[tar_v+5][ep1])
arange = np.array([a_1, a_2, a_3, a_4, a_5, a_6, a_7, a_8, a_9, a_10, a_11])
arange_std = np.array([np.nanstd(a_1), np.nanstd(a_2), np.nanstd(a_3), np.nanstd(a_4),
                       np.nanstd(a_5), np.nanstd(a_6), np.nanstd(a_7), np.nanstd(a_8),
                       np.nanstd(a_9), np.nanstd(a_10), np.nanstd(a_11)])


#b velocity channels, 1 degree diameter    
b_1 = (img1[tar_v-5][ep2])
b_2 = (img1[tar_v-4][ep2])
b_3 = (img1[tar_v-3][ep2])
b_4 = (img1[tar_v-2][ep2])
b_5 = (img1[tar_v-1][ep2])
b_6 = (img1[tar_v][ep2])
b_7 = (img1[tar_v+1][ep2])
b_8 = (img1[tar_v+2][ep2])
b_9 = (img1[tar_v+3][ep2])
b_10 = (img1[tar_v+4][ep2])
b_11 = (img1[tar_v+5][ep2])
brange = np.array([b_1, b_2, b_3, b_4, b_5, b_6, b_7, b_8, b_9, b_10, b_11])
brange_std = np.array([np.nanstd(b_1), np.nanstd(b_2), np.nanstd(b_3), np.nanstd(b_4), 
                       np.nanstd(b_5), np.nanstd(b_6), np.nanstd(b_7), np.nanstd(b_8), 
                       np.nanstd(b_9), np.nanstd(b_10), np.nanstd(b_11)])

med_stda = np.nanmedian(arange_std)
med_stdb = np.nanmedian(brange_std)
arange_std, med_stda 

#arange_std: stdev of 30arcmin region at the location of the galaxy through +/- 5 velocity channels from the central velocity
#med_stda: the median of this list of standard deviations
rowa = objname, '%.7f'%(med_stda)
rowb = arange_std
med_stda

# Table

In [ ]:
#tba = Table(names=('GalaxyName', 'med_std'), 
#             meta={'name': 'first table'},
#              dtype=('<U11', 'f8'))
tba.add_row(rowa)

#tbb = Table(names=('v_-5', 'v_-4', 'v_-3', 'v_-2', 
#                      'v_-1', 'v_0', 'v_+1', 'v_+2', 'v_+3', 'v_+4', 'v_+5'), 
#             meta={'name': 'first table'},
#              dtype=('f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8'))
tbb.add_row(rowb)

#tb = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/hi4pi_resolved_undetected_table.txt', format='ascii')
#tb.add_row(rowd)

tb = (hstack([tba, tbb]))

f = open('galfa_unresolved_undetected_range.txt', 'w')
table = tb
f.write(tabulate(tb))
f.close

tb


# MHI

In [ ]:
def dist(objname):
    tbm = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/dwarfgalcomplete_newnew.txt', format='ascii')
    for i, iname in enumerate(tbm['GalaxyName']):
        if iname == objname:
            find_it = True
            break 
        else: 
            find_it = False
        
    if find_it == True:
        dist = float(tbm['dist(kpc)'][i])
        return dist 

print(dist(objname), objname)

In [ ]:
# MHI = (2.36x10**5)(D**2 mpc)(Stot) for unresolved
x = 2.36*1e5
dmpc = dist(objname)/1e3  #mpc 
tflux = 0.030336279 #put in by hand because every table different/not exactly formatted
stot = tflux*10 #tflux always multiplied by 10km/s

#galfa: 9.1 k/Jy hi4pi: .6 jy/k
g = 1/9.1 
h = .6


MHI = x*(dmpc**2)*(stot*g) #multiply by 'h' or 'g' (hi4pi or galfa) to convert to janskys 
y = MHI
'%.5f'%(y)  #show 5 decimal points

In [ ]:
x = plt.scatter(vel_bins, flux_stds)
plt.show(x)

# Plot:

In [ ]:
plt.figure(figsize=(8, 8))
im = plt.imshow(img1[tar_v, :, :], cmap='jet', origin='lower' , vmin=-.5, vmax=.5)
plt.contour(ep1, colors='white', label='Optical Size')
plt.contour(ep2, colors='red', label='30 Arcmin region')
plt.text(tar_x+15, tar_y+15, objname, color='w', fontsize=14, weight='semibold')
plt.title(objname)
plt.colorbar(im)

#plt.xlim(0, 50)
#plt.ylim(75, 125)

#plt.savefig('/Users/amalyajohnson/Desktop/unresolved undetected/%s_image1_hi4pi.pdf'%(objname)) #to save file
plt.show()